In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
import attr
from twisted.internet import reactor
from txdbus import client as txdbus_client

from pyxsession.config import load_config
from pyxsession.dbus import Service
from pyxsession.executor import Executor, DBusExecutor
from pyxsession.open import ApplicationFinder, exec_key_fields
from pyxsession.urls import UrlRegistry
from pyxsession.xdg.applications import ApplicationsRegistry
from pyxsession.xdg.mime import MimeRegistry


In [3]:
config = load_config()

applications = ApplicationsRegistry(config)
mime = MimeRegistry(config, applications)
urls = UrlRegistry(config, applications)
finder = ApplicationFinder(urls, mime)

app = finder.get_by_url_or_file('https://www.google.com')

conn = await txdbus_client.connect(reactor)

service = Service('org.jfhbrook.pyxsession')

executor = DBusExecutor(service)

underlying = Executor(applications=applications)


In [4]:
await executor.start_server(conn, underlying)

await executor.start_client(conn)

In [5]:
await executor.run_xdg_application(app, exec_key_fields=exec_key_fields(app, 'https://www.google.com'))